# 2-Table Assembling

This notebook assembles the results of the [scraping](https://github.com/njparker1993/oscars_predictions/blob/master/scraping.ipynb) into one dataframe into a machine learning-ready DataFrame.

In [57]:
import pandas as pd
import numpy as np
import re

In [69]:
#nominated and winners of best picture award Oscars from 1927 (year, title, wiki, winner(T/F))
osc_scrape = pd.read_csv('./data/scraping_results/osc_bp.csv')
osc_scrape['film'] = osc_scrape['film'].apply(lambda x: x.strip())
#oscar nominations of award winning films (year, film, link, liczba nominacji, film_text)
noms = pd.read_csv('./data/scraping_results/noms.csv')
#like osc_scrape but for golden globes from 1943
gg_drama = pd.read_csv('./data/scraping_results/gg_drama.csv')

gg_drama['film'] = gg_drama['film'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
gg_drama['film'] = gg_drama['film'].apply(lambda x: x.strip())
#like osc_scrape but for golden globes from 1951
gg_comedy = pd.read_csv('./data/scraping_results/gg_comedy.csv')

gg_comedy['film'] = gg_comedy['film'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
gg_comedy['film'] = gg_comedy['film'].apply(lambda x: x.strip())
#producers guild year film wiki winner from 1989
pga = pd.read_csv('./data/scraping_results/pga.csv')

pga['film'] = pga['film'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
pga['film'] = pga['film'].apply(lambda x: x.strip())
#from 1960
bafta = pd.read_csv('./data/scraping_results/bafta.csv')

bafta['film'] = bafta['film'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
bafta['film'] = bafta['film'].apply(lambda x: x.strip())
#directors guild from 1948
dga = pd.read_csv('./data/scraping_results/dgas.csv')

dga['film'] = dga['film'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
dga['film'] = dga['film'].apply(lambda x: x.strip())
#screen actors guild from 1995
sag = pd.read_csv('./data/scraping_results/sag_ensemble.csv')

sag['film'] = sag['film'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
sag['film'] = sag['film'].apply(lambda x: x.strip())

# Join on Nominations

In [70]:
# I use this order and manually fill in the nominations for this year
noms_merge = noms[['film','nominations']].drop_duplicates()
osc_scrape = pd.merge(osc_scrape, noms_merge, on = 'film', how = 'left')
osc_to_avg = osc_scrape.copy()
avg_noms = int(osc_to_avg.loc[(~osc_to_avg['nominations'].isna())]['nominations'].mean())
osc_scrape.loc[(osc_scrape['nominations'].isna()), 'nominations'] = avg_noms

## Join on the other awards shows

In [62]:
def table_assemble(main_df, to_add_df, show_name):
    """
    Given an awards show scraped from Wikipedia, this function
    Adds it as a feature as a 0/1 flag if it was
    Nominated for that Award and if it won
    """
    nom_col = str('nom_') + show_name
    win_col = str('winner_') + show_name
    # Initalize Columns as no
    main_df[nom_col] = 0
    main_df[win_col] = 0
    to_add_df.columns = ['year','film','wiki','winner_add']
    main_df = pd.merge(main_df, to_add_df[['film','winner_add']], on = 'film', how = 'left')
    main_df.loc[(~main_df['winner_add'].isna()), nom_col] = 1
    main_df.loc[(main_df['winner_add'] == True), win_col] = 1
    main_df = main_df.drop('winner_add', axis = 1)
    
    return main_df


In [71]:
# Adding a response columns
osc_scrape['Oscar_win'] = 0
osc_scrape.loc[(osc_scrape['winner'] == True), 'Oscar_win'] = 1

In [72]:
# Adding on the results of each festival
scraped_dfs = [gg_drama, gg_comedy, pga, bafta, dga, sag]
scraped_names = ['gg_drama','gg_comedy','pga', 'bafta', 'dga', 'sag']

for i in range(len(scraped_dfs)):
    osc_scrape = table_assemble(osc_scrape, scraped_dfs[i], scraped_names[i])

In [73]:
osc_scrape[osc_scrape['year']>1995]

,year,film,wiki,winner,nominations,Oscar_win,nom_gg_drama,winner_gg_drama,nom_gg_comedy,winner_gg_comedy,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag
402,1996,The English Patient,/wiki/The_English_Patient_(film),True,12.0,1,1,1,0,0,1,1,1,1,1,1,1,0
403,1996,Fargo,/wiki/Fargo_(1996_film),False,7.0,0,0,0,1,0,1,0,1,0,1,0,0,0
404,1996,Jerry Maguire,/wiki/Jerry_Maguire,False,5.0,0,0,0,1,0,0,0,0,0,1,0,0,0
405,1996,Secrets & Lies,/wiki/Secrets_%26_Lies_(film),False,7.0,0,1,0,0,0,0,0,1,0,1,0,0,0
406,1996,Shine,/wiki/Shine_(film),False,7.0,0,1,0,0,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2022,The Fabelmans,/wiki/The_Fabelmans,False,7.0,0,1,1,0,0,1,0,0,0,1,0,1,0
592,2022,Tár,/wiki/T%C3%A1r,False,7.0,0,1,0,0,0,1,0,1,0,1,0,0,0
593,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,6.0,0,1,0,0,0,1,0,0,0,1,0,0,0
594,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,7.0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [74]:
# Some awards didn't exist until certain years
for i in range(len(scraped_dfs)):
    print(scraped_names[i], scraped_dfs[i].year.min())

gg_drama 1943
gg_comedy 1951
pga 1989
bafta 1960
dga 1948
sag 1995


In [75]:
# Saving the data
osc_scrape.to_csv('./data/processed_results/osc_df')

# Results
The DataFrame is now Machine Learning ready, with 0's and 1's as markers for how a given Oscar-Nominated Film did for that Awards show. The response column will be Oscar_win. An example of this final DataFrame can be seen below.

In [76]:
cols = list(osc_scrape.columns)
display_cols = cols[0:2] + cols[4:]
osc_scrape[display_cols].sample(5)

,year,film,nominations,Oscar_win,nom_gg_drama,winner_gg_drama,nom_gg_comedy,winner_gg_comedy,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag
361,1987,Moonstruck,6.0,0,0,0,1,0,0,0,0,0,0,0,0,0
199,1955,Picnic,6.0,0,0,0,0,0,0,0,0,0,1,0,0,0
268,1969,Anne of the Thousand Days,10.0,0,1,1,0,0,0,0,0,0,0,0,0,0
553,2018,The Favourite,10.0,0,0,0,1,1,1,0,1,0,0,0,0,0
515,2014,Birdman,7.0,1,0,0,1,0,1,1,1,0,1,1,1,1


In [77]:
len(osc_scrape)

596